In [ ]:
#workdir_ = '/glade/work/juliob/'
import sys
#######################################
# Leave this for now. But it should change to better
# method as here:
import os
This_module_path = os.getcwd()  #os.path.dirname(os.path.abspath(__file__))
workdir_ = os.path.join(This_module_path, '../../' )
# sys.path.append(utils_path)
# print( f" a path added in {__name__} {utils_path} ")

print( f" In {__name__} we have This_module_path={This_module_path} " )
print( f" In {__name__} we have workdir_={workdir_} " )

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime



########################################
# Now import your own stuff
########################################

#From in here
import gw_common as GWc
import gw_movmtn as GWmm

sys.path.append(workdir_ + 'myPythonTools/Utils/')
import AveragingUtils as Av
import PlotUtil as Pu
import utils as uti
import validation_data as Val
import shr_const
from shr_const import ShrConst as Cs

sys.path.append(workdir_ + 'myPythonTools/Plotting/')
import LatLonMaps as LL


sys.path.append(workdir_ + 'PyRegridding/Regridder/')
import var_A_x_B as vAB

sys.path.append(workdir_ + 'PyRegridding/Utils/')
import MakePressures as MkP
import GridUtils as GrU



importlib.reload( GWmm )
importlib.reload( GWc )
importlib.reload( LL )
importlib.reload( uti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
importlib.reload(vAB)
importlib.reload(MkP)


In [ ]:
from datetime import datetime, timedelta
import calendar

def day_of_year_to_date(year, day_of_year):
    # Create a date object for January 1st of the given year
    jan_first = datetime(year, 1, 1)
    # Add the day_of_year to this date (subtract 1 because day_of_year is 1-based)
    target_date = jan_first + timedelta(days=day_of_year - 1)
    return target_date


In [ ]:

year=2010
month=10

#timetag = [ f'{year:04d}-{month:02d}-*', f'{year:04d}-{month+1:02d}-*', ]
timetag = [ f'{year:04d}-{month:02d}-*', ] 

expA = "c160mm_ne30pg3_FMTndg_cciasi_x05"
C1 = uti.MakeDict4Exp( exp=expA ,
                     user='juliob',
                     subd='hist',  #cfg[x_]['grid'] , 
                     hsPat='cam.h1i',
                     ymdPat= timetag ,
                     verbose=True, open_dataset=True )

C2 = uti.MakeDict4Exp( exp=expA ,
                     user='juliob',
                     subd='hist',  #cfg[x_]['grid'] , 
                     hsPat='cam.h1i.TAU_xpwp_SV_b',
                     ymdPat= timetag ,
                     verbose=True, open_dataset=True )


In [ ]:
print(list(C1.X.variables))
print(list(C2.X.variables))


In [ ]:
#####################################################################
# Subselecting SH high-lats
#####################################################################
latitude_range = (-90, -45)  # Example latitude range

latitude_mask = np.where(
    (C1.X['lat'].values >= latitude_range[0]) & (C1.X['lat'].values <= latitude_range[1]) )


print( len(latitude_mask[0]))
# Select data using these indices
#selected_data = ds.isel(ncol=latitude_indices)

latitude_indices=latitude_mask[0]

X1 = C1.X.isel(ncol=latitude_indices )
X2 = C2.X.isel(ncol=latitude_indices )




In [ ]:
#plt.plot( C1.X.TAU_MOVMTN.values[10,:,:].flatten())
#plt.plot( C1.X.TAU_MOVMTN.values[10,80,:].flatten() - C2.X.TAU_redo.values[10,80,:].flatten()     )
#plt.plot( X2.TAU_redo.values[10,50,:]    )
plt.scatter( X2.TAU_redo.values[10,30,:] , X1.TAU_MOVMTN.values[10,30,:]    )


In [ ]:
import glob

# Generate the list of actual file paths for each pattern
file_paths_nov = glob.glob('/glade/derecho/scratch/juliob/archive/c160mm_ne30pg3_FMTndg_cciasi_x05/atm/hist/c160mm_ne30pg3_FMTndg_cciasi_x05.cam.h1i.2010-11-*.nc')
file_paths_dec = glob.glob('/glade/derecho/scratch/juliob/archive/c160mm_ne30pg3_FMTndg_cciasi_x05/atm/hist/c160mm_ne30pg3_FMTndg_cciasi_x05.cam.h1i.2010-12-*.nc')

# Combine the lists of file paths
all_file_paths = file_paths_nov + file_paths_dec

# Open multiple files into a single xarray dataset
BooBoo = xr.open_mfdataset(all_file_paths, combine='by_coords')

# Check the dataset
print(BooBoo)


In [ ]:
BooBoo.time

In [ ]:
CC=xr.open_dataset('/glade/campaign/cgd/amp/juliob/amwg_dev/Concordiasi/cciasi_flux_6h.nc')

ntimes = len( CC.time )
print(ntimes)

CCdates = ['']*ntimes
for n in np.arange( ntimes ):
    date=day_of_year_to_date(2010, CC.time[n].values)
    seconds = date.hour * 3600 
    CCdates[n] = date.strftime("%Y-%m-%d")+f"-{seconds:05}"


In [ ]:


days_in_month = calendar.monthrange(year, month)[1]
days_in_month = days_in_month + calendar.monthrange(year, month+1)[1]

toot = timetag[0].replace('*', f"01-00000")
try:
    start_index = CCdates.index(toot)
    stop_index = start_index + 4*days_in_month
except ValueError:
    print(f"'{toot}' is not in the data.")

print(start_index,CCdates[start_index],days_in_month,stop_index )

In [ ]:
CC=CC.isel(nt=np.arange(start=start_index ,stop=stop_index ) )
CC=CC.isel(nlat=np.arange(start=1,stop=20) )
CC= CC.assign_coords(longitude=((CC.longitude + 360) % 360)).sortby('longitude')

In [ ]:
CX = C1.X   #.isel(time=np.arange(609))
CXv = C1v.X   #.isel(time=np.arange(609))
CXu = C1u.X   #.isel(time=np.arange(609))


In [ ]:
nobs = CC.number_of_observations.values
amf =CC.absolute_momentum_flux.values
mmf = CX.TAU_MOVMTN.values
mmfv = CXv.TAU_redo.values
mmfu = CXu.TAU_redo.values
ogmmf=np.sqrt( CX.TAUARDGBETAX.values**2 + CX.TAUARDGBETAY.values**2 )
print(amf.shape)

In [ ]:
zlev=  -7.0 * np.log( CX.lev.values / 1_000. )
print(CX.lev[42].values)
print(zlev[42])

In [ ]:
nobo=np.where( nobs>10, 1., 0.)


In [ ]:
kk=42
mmf18=mmf[:,kk,:,:]
mmfv18=mmfv[:,kk,:,:]
mmfu18=mmfu[:,kk,:,:]
ogmmf18=ogmmf[:,kk,:,:]
mmf18.shape


In [ ]:
bobo = np.where( ( (amf*nobo>0.1)&(1000*(mmfv18+mmf18)*nobo<0.1 )), 1., 0. )

In [ ]:
#plt.contourf( CCor.longitude, CCor.latitude, np.average( amf*nobo,axis=0),levels = np.linspace(0.,.5,num=21) )


fields=[  np.sum( amf*nobo,axis=0)/(np.sum(nobo,axis=0)+1e-6)   , 1000.*np.sum( nobo*(mmfv18),axis=0) /(np.sum(nobo,axis=0)+1e-6)  , 1000.*np.sum( nobo*(mmf18),axis=0) /(np.sum(nobo,axis=0)+1e-6)   ,np.sum( nobo,axis=0) ] 
clevs= [np.linspace(0.,10,num=21) , np.linspace(0.,10,num=21) , np.linspace(0.,10,num=21) , np.linspace(0.,30,num=31) ]
titles=['CONCORDIASI abs mom flux (AMF) at 18km', '500hPa Vorticity source AMF, 18km ', 'PBL CLUBB mom-flux source AMF, 18km', 'CONCORDIASI # Obs']
LL.Maps (  fields=fields,lats=[CC.latitude],lons=[CC.longitude],clevs= clevs ,titles=titles)


In [ ]:
#plt.contourf( CCor.longitude, CCor.latitude, np.average( amf*nobo,axis=0),levels = np.linspace(0.,.5,num=21) )


fields=[  np.sum( amf*bobo,axis=0)/(np.sum(bobo,axis=0)+1e-6)   , 1000.*np.sum( bobo*(mmfu18),axis=0) /(np.sum(bobo,axis=0)+1e-6)  , 1000.*np.sum( bobo*(mmf18),axis=0) /(np.sum(bobo,axis=0)+1e-6)   ,np.sum( bobo,axis=0) ] 
clevs= [np.linspace(0.,10,num=21) , np.linspace(0.,10,num=21) , np.linspace(0.,10,num=21) , np.linspace(0.,15,num=31) ]
titles=['CONCORDIASI abs mom flux', 'OGW abs momflux', 'Moving mtn flux', 'CONCORDIASI # Obs']
LL.Maps (  fields=fields,lats=[CC.latitude],lons=[CC.longitude],clevs= clevs ,titles=titles)


In [ ]:
print( np.shape(amf) )
print(CX.lon[108].values,CX.lat[14].values,CX.lat[9].values)
plt.scatter(   (amf*nobo)[ :,9:14, 0:108 ].flatten() , 1000*( (mmfv18+mmf18 )*nobo)[ :,9:14,0:108].flatten()    )
plt.xlim(0,50)

In [ ]:
print( np.shape(amf) )
print(CX.lon[108].values,CX.lat[14].values,CX.lat[9].values)
xdat=  (amf*nobo)[ :,9:14, 0:108 ].flatten() 
ydat=  1000*( (mmfv18+mmf18 )*nobo)[ :,9:14,0:108].flatten()    
xbe=np.linspace( 1., 50., num=51 )
ybe=np.linspace( 1., 20., num=21 )

H, x_edges, y_edges = np.histogram2d(xdat, ydat, bins=[xbe, ybe])

plt.imshow(H.T, origin='lower', aspect='auto', 
           extent=[x_edges[0], x_edges[-1], y_edges[0], y_edges[-1]],
           cmap='viridis')
plt.colorbar(label='Counts')
plt.xlabel('X axis')
plt.ylabel('Y axis')
plt.title('2D Joint Histogram')


In [ ]:
print( np.shape(amf) )

plt.scatter(   (amf*bobo)[ :,9:14, 0:120 ].flatten() , 1000*( (mmfv18+mmf18 )*bobo)[ :,9:14,0:120].flatten()    )
plt.xlim(0,50)

In [ ]:
levels=np.linspace(0,30,num=31)
fig,ax=plt.subplots(1,2 , figsize=(14,7) )
ax[0].contourf( np.sum(nobo,axis=0) ,levels=levels/2)
ax[1].contourf( np.sum(bobo,axis=0) ,levels=levels/5)


In [ ]:
levels=np.linspace(0,10,num=11)
fig,ax=plt.subplots(1,2 , figsize=(14,7) )
ax[0].contourf( 1000.*np.sum( bobo*(mmf18),axis=0) /(np.sum(bobo,axis=0)+1e-6) ,levels=levels )
ax[0].set_xlim(70,100)
#plt.contourf( 1000.*np.sum( nobo*(ogmmf18),axis=0) /(np.sum(nobo,axis=0)+1e-6) )
ax[1].contourf( 1.*np.sum( bobo*(amf),axis=0) /(np.sum(bobo,axis=0)+1e-6) ,levels=levels )
ax[1].set_xlim(70,100)


In [ ]:
plt.plot( (bobo*amf)[:,17,82] )
plt.xlim(180,220)



In [ ]:
print(CX.date[199].values)
print(CX.time[199].values)


In [ ]:
#plt.contourf( CCor.longitude, CCor.latitude, np.average( amf*nobo,axis=0),levels = np.linspace(0.,.5,num=21) )

Xplo=CXu
#fields=[  (amf*nobo)[199,:,:]  , 1000.*( nobo*mmfu18)[199,:,:]  ] 
fields=[  (amf*nobo)[199,:,:]  , 1000.*( Xplo.TAU_redo[:,90,:,:].values)[199,:,:], 1000.*( Xplo.TAU_redo[:,80,:,:].values)[199,:,:], 1000.*( Xplo.TAU_redo[:,70,:,:].values)[199,:,:] ,1000.*( Xplo.TAU_redo[:,60,:,:].values)[199,:,:] ,1000.*( Xplo.TAU_redo[:,40,:,:].values)[199,:,:]  ] 

clevs= [np.linspace(0.,10,num=21) , np.linspace(0.,10,num=21) , np.linspace(0.,10,num=21) , np.linspace(0.,30,num=31) ]
titles=['CONCORDIASI abs mom flux (AMF) at 18km', '500hPa Vorticity source AMF, 18km ', 'PBL CLUBB mom-flux source AMF, 18km', 'CONCORDIASI # Obs']
LL.Maps (  fields=fields,lats=[CC.latitude],lons=[CC.longitude],clevs= clevs ,titles=titles)


In [ ]:
CC

In [ ]:
#plt.contourf( CCor.longitude, CCor.latitude, np.average( amf*nobo,axis=0),levels = np.linspace(0.,.5,num=21) )


#fields=[  (amf*nobo)[199,:,:]  , 1000.*( nobo*CXu.TAU_redo[:,90,:,:].values)[199,:,:]  ] 
fields=[  (amf*nobo)[199,:,:]  , 1000.*( CXu.TAU_redo[:,40,:,:].values)[199,:,:]  ] 
clevs= [np.linspace(0.,10,num=21) , np.linspace(0.,10,num=21) , np.linspace(0.,10,num=21) , np.linspace(0.,30,num=31) ]
titles=['CONCORDIASI abs mom flux (AMF) at 18km', '500hPa Vorticity source AMF, 18km ', 'PBL CLUBB mom-flux source AMF, 18km', 'CONCORDIASI # Obs']
#LL.Maps_NoProj (  fields=fields,lats=[np.arange(19) ],lons=[np.arange(144) ],clevs= clevs ,titles=titles)
LL.Maps_NoProj (  fields=fields,lats=[CC.latitude],lons=[CC.longitude],clevs= clevs ,titles=titles)
